## Part 1: Preprocessing

In [130]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
display(attrition_df.head())
display(attrition_df['OverTime'])

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


0       Yes
1        No
2       Yes
3       Yes
4        No
       ... 
1465     No
1466     No
1467    Yes
1468     No
1469     No
Name: OverTime, Length: 1470, dtype: object

In [131]:
# Determine the number of unique values in each column
attrition_df.nunique()

Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [132]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]
print(y_df['Department'].unique())
print(y_df['Attrition'].unique())

['Sales' 'Research & Development' 'Human Resources']
['Yes' 'No']


In [133]:
# Create a list of at least 10 column names to use as X data
selected_columns = ['Age', 'Education', 'EducationField', 'EnvironmentSatisfaction',  'JobLevel', 'JobRole', 'JobSatisfaction',  'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']

# Create X_df using your selected columns
X_df = attrition_df[selected_columns]

# Show the data types for X_df
display(X_df.dtypes)

Age                          int64
Education                    int64
EducationField              object
EnvironmentSatisfaction      int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
NumCompaniesWorked           int64
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StockOptionLevel             int64
TotalWorkingYears            int64
TrainingTimesLastYear        int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsInCurrentRole           int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
dtype: object

In [134]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df)

In [135]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary

# Create a mapping dictionary for EducationField
education_field_mapping = {field: i for i, field in enumerate(X_train['EducationField'].unique())}

# Convert EducationField using the mapping
X_train['EducationField'] = X_train['EducationField'].map(education_field_mapping)
X_test['EducationField'] = X_test['EducationField'].map(education_field_mapping)

# Convert OverTime to numeric (True/False)
X_train['OverTime'] = X_train['OverTime'].map({'Yes': 1, 'No': 0})
X_test['OverTime'] = X_test['OverTime'].map({'Yes': 1, 'No': 0})

# Convert JobRole to numeric using same approach as EducationField
job_role_mapping = {role: i for i, role in enumerate(X_train['JobRole'].unique())}
X_train['JobRole'] = X_train['JobRole'].map(job_role_mapping)
X_test['JobRole'] = X_test['JobRole'].map(job_role_mapping)


In [136]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [137]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
y_department_train_encoded = department_encoder.transform(y_train[['Department']])
y_department_test_encoded = department_encoder.transform(y_test[['Department']])

y_department_train_encoded_output_shape = y_department_train_encoded.shape[1]
 
print("Department output shape: ", y_department_train_encoded_output_shape)
display(y_department_train_encoded)



Department output shape:  3


array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]])

In [138]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
y_attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])

y_attrition_train_encoded_output_shape = y_attrition_train_encoded.shape[1]

print("Attrition output shape: ", y_attrition_train_encoded_output_shape)
display(y_attrition_train_encoded)


Attrition output shape:  2


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

## Part 2: Create, Compile, and Train the Model

In [139]:
# Find the number of columns in the X training data.
num_features = X_train.shape[1]

# Create the input layer
input_layer = layers.Input(shape=(num_features,))

# Create at least two shared layers
drop_layers = layers.Dense(units=64, activation='relu', name='drop_layer1')(input_layer)
drop_layers = layers.Dropout(0.2)(drop_layers)
drop_layers = layers.Dense(units=128, activation='relu', name='drop_layer2')(drop_layers)
drop_layers = layers.Dropout(0.2)(drop_layers)

shared_layer1 = layers.Dense(units=64, activation='relu', name='shared_layer1')(drop_layers)
shared_layer2 = layers.Dense(units=128, activation='relu', name='shared_layer2')(shared_layer1)


In [140]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = layers.Dense(units=64, activation='relu', name='department_branch')(shared_layer2)

# Create the output layer
department_output = layers.Dense(units=y_department_train_encoded_output_shape, activation='softmax', name='department_output')(department_branch)

In [141]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = layers.Dense(units=64, activation='relu', name='attrition_branch')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(units=y_attrition_train_encoded_output_shape, activation='sigmoid', name='attrition_output')(attrition_branch)


In [142]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])    

# Compile the model
model.compile(
            optimizer='adam',
            loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'binary_crossentropy'}, 
            metrics={'department_output':['accuracy'], 'attrition_output':['accuracy']}
            )

# Summarize the model
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7       │ (None, 20)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ drop_layer1 (Dense) │ (None, 64)        │      1,344 │ input_layer_7[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_14          │ (None, 64)        │          0 │ drop_layer1[0][0] │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ drop_layer2 (Dense) │ (None, 128)       │      8,320 │ dropout_14[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_15          │ (None, 128)       │          0 │ drop_layer2[0][0] │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer1       │ (None, 64)        │      8,256 │ dropout_15[0][0]  │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ shared_layer2       │ (None, 128)       │      8,320 │ shared_layer1[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_branch   │ (None, 64)        │      8,256 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_branch    │ (None, 64)        │      8,256 │ shared_layer2[0]… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ department_output   │ (None, 3)         │        195 │ department_branc… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_output    │ (None, 2)         │        130 │ attrition_branch… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 43,077 (168.27 KB)

 Trainable params: 43,077 (168.27 KB)

 Non-trainable params: 0 (0.00 B)

In [143]:
# Train the model
model.fit(X_train_scaled, [y_department_train_encoded, y_attrition_train_encoded], epochs=100)

Epoch 1/100


35/35 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - attrition_output_accuracy: 0.6543 - attrition_output_loss: 0.6125 - department_output_accuracy: 0.5341 - department_output_loss: 0.9062 - loss: 1.5185
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8246 - attrition_output_loss: 0.4565 - department_output_accuracy: 0.6699 - department_output_loss: 0.7296 - loss: 1.1864
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8220 - attrition_output_loss: 0.4366 - department_output_accuracy: 0.7328 - department_output_loss: 0.6621 - loss: 1.0991
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8290 - attrition_output_loss: 0.4117 - department_output_accuracy: 0.7787 - department_output_loss: 0.6065 - loss: 1.0182 
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8587 - attrition_output_loss: 0.3699 - department_output_accuracy: 0.7587 - department_output_loss: 0.5962 - loss

In [144]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, [y_department_test_encoded, y_attrition_test_encoded])

12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - attrition_output_accuracy: 0.8486 - attrition_output_loss: 0.4193 - department_output_accuracy: 0.8907 - department_output_loss: 0.2984 - loss: 0.7160  


In [147]:
# Print the accuracy for both department and attrition
print("Results: ", results)
print(f"Department accuracy: {results[3]:.2%}")
print(f"Attrition accuracy: {results[4]:.2%}")

Results:  [0.8175114989280701, 0.3927275240421295, 0.43603548407554626, 0.85326087474823, 0.8586956262588501]
Department accuracy: 85.33%
Attrition accuracy: 85.87%


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. No accuracy is not the best metric. For attrition the data imbalance is to great. The model could just say everyone is a no and still get the same accuracy. For department it negelects a few of the values. You need to show more of the results and how things are connected to know what is really happening.  
2. I chose sigmoid for the attrition branch. It only has true or false. Then I chose softmax for the Department branch. This one is better with the increased number of output options. It alos ensures that there is only 1 of the options selected.
3. We could change out the scalers. Have better evaluation metrics. We could even use Hyper Parameter Optimizaion.